# convolutional neural network

In [2]:
from __future__ import print_function, division, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
# paramters
learning_rate = 0.001
n_steps = 2000
batch_size = 128
display_steps = 50

# network parameters
d_input = 784
n_classes = 10
dropout = 0.25

In [6]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    # define a scope for reusing the variables
    with tf.variable_scope('Convnet', reuse=reuse):
        # tf estimator input is a dict, in case of multiple inputs
        x = x_dict['images']
        
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, [-1, 28, 28, 1])
        
        conv1 = tf.layers.conv2d(x, filters=32, kernel_size=5, activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=3, activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(pool2)
        
        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, n_classes)
    return out

In [10]:
def model_fn(features, labels, mode):
    logits_train = conv_net(features, n_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, n_classes, dropout, reuse=True, is_training=False)
    
    # prediction
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_prob = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # define loss and optimizer
    loss_op = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=tf.cast(labels, dtype=tf.int32))
    )
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estimator_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op}
    )
    
    return estimator_specs

In [11]:
# build the estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvt4nw_iy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3bfe2c4780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images},
    y=mnist.train.labels,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True
)

model.train(input_fn, steps=n_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvt4nw_iy/model.ckpt.
INFO:tensorflow:loss = 2.3067966, step = 1
INFO:tensorflow:global_step/sec: 21.2689
INFO:tensorflow:loss = 0.20670016, step = 101 (4.702 sec)
INFO:tensorflow:global_step/sec: 22.2047
INFO:tensorflow:loss = 0.09053618, step = 201 (4.503 sec)
INFO:tensorflow:global_step/sec: 22.5966
INFO:tensorflow:loss = 0.04310241, step = 301 (4.425 sec)
INFO:tensorflow:global_step/sec: 22.1981
INFO:tensorflow:loss = 0.08340771, step = 401 (4.505 sec)
INFO:tensorflow:global_step/sec: 22.1271
INFO:tensorflow:loss = 0.030523479, step = 501 (4.519 sec)
INFO:tensorflow:global_step/sec: 22.2469
INFO:tensorflow:loss = 0.

In [ ]:
# evaluate the model
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images},
    y=mnist.test.labels,
    batch_size=batch_size,
    shuffle=False
)

model.evaluate(input_fn)